# Docker Build Your Own Container{background-color="black" background-image="images/bob.jpg" background-size="100%" background-opacity="0.5"}


## Let's start with an example
::: {.fragment}
### Node Bulletin

```bash
git clone https://github.com/dockersamples/node-bulletin-board
cd node-bulletin-board/bulletin-board-app
```

:::

### Let's see the Docker File

```dockerfile
FROM node:current-slim

WORKDIR /usr/src/app
COPY package.json .
RUN npm install

EXPOSE 8080
CMD [ "npm", "start" ]

COPY . .

```

### Build Image
```bash
docker image build -t bulletinboard:1.0 .
```

## Run your image as a container
```bash
docker container run --publish 8000:8080 --rm --name bb bulletinboard:1.0
```

## Is running ?
```bash
docker ps
```

### Delete it
```bash
docker container rm --force bb
```

## Try your self

Go to [Docker Sample](https://github.com/dockersamples/)

Choose an example and try to run it

## Build an image using Dockerfile
[NicsMeme](https://imgflip.com/i/50yk9z) inspired by [Docker Anti Patterns](https://codefresh.io/containers/docker-anti-patterns/)

![](https://i.imgflip.com/50yk9z.jpg){.r-stretch}


### Build command
<https://docs.docker.com/build/>

::::{.columns}

::: {.fragment .column width="50%"}
![](images/DockerBuild.png)
:::

::: {.fragment .column width="50%"}

Run the cli command ```docker build```

- without arguments search for a Dockerfile in the same directory
- -f option  specify the path of the docker file ```bash $ docker build -f /path/to/a/Dockerfile ```
- -t or --tag define a name for the image
:::

::::




### Syntax 
<https://docs.docker.com/reference/dockerfile/>

- Docker can build images automatically by reading the instructions from a Dockerfile. 
- A Dockerfile is a text document that contains all the commands a user could call on the command line to assemble an image.

::: {.fragment}
```DockerFile
# Comment
INSTRUCTION arguments
```

*Convention: instructions are in UPPER case*
:::

#### FROM
<https://docs.docker.com/engine/reference/builder/#from>

```DockerFile
FROM [--platform=<platform>] <image>[:<tag>] [AS <name>]
```

:::: {.columns}

::: {.fragment .column width="50%"}
- First instruction after comments/parser directives/arg
- Indicates the Image source (from a registry)
- Can use ARG parameter 
- it's your baby!
::: 

::: {.fragment .column width="50%"}

![](images/babyboss.png)

:::
::::







#### ARG
<https://docs.docker.com/engine/reference/builder/#arg>

:::: {.columns}

::: {.fragment .column width="70%"}

Define enviroment variabile used in the Builder

```DockerFile
ARG <name>[=<default value>]
```    

Argument can be used in DockerFile FROM, RUN, ...
also passed during the BUILD

```bash
$ docker build --build-arg user=what_user 
```

ARG are referenced using standard ${VARIABLE} notation

Be careful with ARG

- scope starts from the line where is defined
- ENV wins on ARG in RUN 
- There are a set of predefined ARG (like HTTP_PROXY)
- Don't pass secret in ARG 

::: 

::: {.fragment .column width="30%"}

![](images/arguments-penauts.png)

:::
::::






#### ENV

<https://docs.docker.com/engine/reference/builder/#env>

:::: {.columns}

::: {.fragment .column width="50%"}

Define enviroment variabile that _persist_ in the container

```DockerFile
ENV <key> <value>
ENV <key>=<value> ...
```    

Enviroment variable can be also passed while starting 

```bash
$ docker run -e "deep=purple"
```

::: 

::: {.fragment .column width="50%"}
![](images/wikipedia-env-variable.png)
:::
::::





#### RUN (shell mode)
<https://docs.docker.com/engine/reference/builder/#env>

:::: {.columns}

::: {.fragment .column width="50%"}

Execute commands in the _layer_ on top of the current image

 (shell form, the command is run in a shell, which by default is /bin/sh -c on Linux or cmd /S /C on Windows)

```DockerFile
RUN <command>
```    

The default shell for the shell form can be changed using the SHELL command.

In the shell form you can use a \ (backslash) to continue a single RUN instruction onto the next line. For example, consider these two lines:

```DockerFile
RUN /bin/bash -c 'source $HOME/.bashrc; \
echo $HOME'
```

::: 

::: {.fragment .column width="50%"}

![](images/docker-layers.png)

:::
::::





#### RUN (exec mode)
<https://docs.docker.com/engine/reference/builder/#run>

::: {.fragment}

```DockerFile
RUN ["executable", "param1", "param2"] (exec form)
```    

Unlike the shell form, the exec form does not invoke a command shell. This means that normal shell processing does not happen. 

:::

::: {.fragment}
> To use a different shell, other than ‘/bin/sh’, use the exec form passing in the desired shell. For example, 

```DockerFile
RUN ["/bin/bash", "-c", "echo hello"]
```
:::

::: {.fragment}
For example, `RUN [ "echo", "$HOME" ]` will not do variable substitution on $HOME. If you want shell processing then either use the shell form or execute a shell directly, for example:

```DockerFile
RUN [ "sh", "-c", "echo $HOME" ]. 
```

> The exec form is parsed as a JSON array, which means that you must use double-quotes (“) around words not single-quotes (‘).

:::

## CMD (exec form)

The main purpose of a CMD is to provide defaults for an executing container. These defaults can include an executable, or they can omit the executable, in which case you must specify an ENTRYPOINT instruction as well.

Exec form, this is the preferred form

```DockerFile
CMD ["executable","param1","param2"] 
```    

Like RUN exec form do not invoke a shell, so in order to force the variabile substituion done by shell use

```DockerFile
CMD [ "sh", "-c", "echo $HOME" ]
```

https://docs.docker.com/engine/reference/builder/#env

## CMD (shell form)

The main purpose of a CMD is to provide defaults for an executing container. These defaults can include an executable, or they can omit the executable, in which case you must specify an ENTRYPOINT instruction as well.


```DockerFile
CMD command param1 param2
```    

If you want to run your <command> without a shell then you must express the command as a JSON array and give the full path to the executable. This array form is the preferred format of CMD. Any additional parameters must be individually expressed as strings in the array:
 
```DockerFile
FROM ubuntu
CMD echo "This is a test." | wc -l
```

https://docs.docker.com/engine/reference/builder/#cmd

#### Confused ? 


:::: {.columns}

::: {.fragment .column width="50%"}

![](images/gandalf-docker-cmd.png)

::: 

::: {.fragment .column width="50%"}

- RUN actually runs a command and commits the result
- CMD does not execute anything at build time, but specifies the intended command for the image.
:::
::::



## ENTRYPOINT 
<https://docs.docker.com/reference/dockerfile/#entrypoint>



::::{.columns}

::: {.fragment .column width="50%"}

:::{.callout-note}
An `ENTRYPOINT` allows you to configure a container that will run as an executable.
::: 

- Exec form  ```DockerFile ENTRYPOINT ["executable", "param1", "param2"]```
- Shell form ```DockerFile ENTRYPOINT command param1 param2```
:::

::: {.fragment .column width="50%"}

**Example**

- Run a container from [Official Image](https://hub.docker.com/_/nginx) ```bash docker run -i -t --rm -p 80:80 nginx ```
- Test that it works correctly on the machine
- Try ```bash docker run -i -t --rm -p 80:80 nginx cat docker-entrypoint.sh```
- Let give a look to the [Dockerfile](https://hub.docker.com/layers/library/nginx/latest/images/sha256-396c6e925f28fbbed95a475d27c18886289c2bbc53231534dc86c163558b5e4b?context=explore)

:::



::::






### CMD - ENTRYPOINT interaction
<https://docs.docker.com/reference/dockerfile/#understand-how-arg-and-from-interact>

::::{.columns}

::: {.fragment .column width="50%"}
Both CMD and ENTRYPOINT instructions define what command gets executed when running a container. 

There are few rules that describe their co-operation.

- Dockerfile should specify at least one of CMD or ENTRYPOINT commands.
- ENTRYPOINT should be defined when using the container as an executable.
- CMD should be used as a way of defining default arguments for an ENTRYPOINT command or for executing an ad-hoc command in a container
:::

::: {.fragment .column width="50%"}
![](https://i.imgflip.com/7ewgko.jpg)

:::

::::



# Il cannolo della fortuna {background-color="black" background-image="images/sicilianfortune.jpeg" background-size="100%" background-opacity="0.3"}
A simple fortune extended with sicilian proverbs

## Design

Sources

- https://it.wikiquote.org/wiki/Proverbi_siciliani

::: {.fragment}

**Build** 
```bash
cd fortune
docker build -t tap:fortune .
```
:::

::: {.fragment}

**Code** 

```Dockerfile
FROM ubuntu
RUN apt-get update && apt-get -y install fortunes fortunes fortune-mod
WORKDIR /usr/share/games/fortunes/
ADD sicilian .
RUN strfile -c % sicilian 
ENTRYPOINT ["/usr/games/fortune"]
CMD ["sicilian"]
```
:::

## Demo

In [2]:
%%bash
docker run --rm tap:fortune

Beatu cui pri autru si castiga.


In [8]:
%%bash
docker run --rm tap:fortune computers

Breadth-first search is the bulldozer of science.
		-- Randy Goebel


# Back to Dockerfile Syntax

### EXPOSE
<https://docs.docker.com/reference/dockerfile/#expose>

> The EXPOSE instruction informs Docker that the container listens on the specified network ports at runtime.

```DockerFile
EXPOSE <port> [<port>/<protocol>...]
```    

- The EXPOSE instruction does not actually publish the port. 

- It functions as a type of documentation between the person who builds the image and the person who runs the container, about which ports are intended to be published. 

- To actually publish the port when running the container, use the -p flag on docker run to publish and map one or more ports, or the -P flag to publish all exposed ports and map them to high-order ports.

- ```bash docker run -p 80:80/tcp -p 80:80/udp ```



## ADD
<https://docs.docker.com/reference/dockerfile/#add>

:::{.fragment}
The ADD instruction copies new files, directories or remote file URLs from <src> and adds them to the filesystem of the image at the path <dest>.


```DockerFile
ADD [--chown=<user>:<group>] <src>... <dest>
ADD [--chown=<user>:<group>] ["<src>",... "<dest>"] (this form is required for paths containing whitespace)
```    
:::

:::{.fragment}
The destination is an absolute path, or a path relative to WORKDIR, into which the source will be copied inside the destination container.
    
```DockerFile
ADD test relativeDir/          # adds "test" to `WORKDIR`/relativeDir/
ADD test /absoluteDir/         # adds "test" to /absoluteDir/
```
:::

## COPY

:::{.fragment}
The COPY instruction copies new files or directories from <src> and adds them to the filesystem of the container at the path <dest>.
    
```DockerFile
COPY [--chown=<user>:<group>] <src>... <dest>
COPY [--chown=<user>:<group>] ["<src>",... "<dest>"] (this form is required for paths containing whitespace)
```
:::

![](images/docker-copy-add-meme.png)

## VOLUME
<https://docs.docker.com/reference/dockerfile/#volume>

:::: {.columns}

::: {.fragment .column width="50%"}
The VOLUME instruction creates a mount point with the specified name and marks it as holding externally mounted volumes from native host or other containers.r

```DockerFile
VOLUME ["/data"]
```

The docker run command initializes the newly created volume with any data that exists at the specified location within the base image. 
::: 

::: {.fragment .column width="50%"}

For example, consider the following Dockerfile snippet:

```DockerFile
FROM ubuntu
RUN mkdir /myvol
RUN echo "hello world" > /myvol/greeting
VOLUME /myvol
```

This Dockerfile results in an image that causes docker run to create a new mount point at /myvol and copy the greeting file into the newly created volume.

:::
::::

## WORKDIR
<https://docs.docker.com/reference/dockerfile/#workdir>

```DockerFile
WORKDIR /path/to/workdir
```    

- The WORKDIR instruction sets the working directory for any RUN, CMD, ENTRYPOINT, COPY and ADD instructions that follow it in the Dockerfile.
- If the WORKDIR doesn’t exist, it will be created even if it’s not used in any subsequent Dockerfile instruction.
- Can be used multiple times

:::{.fragment}
```bash
WORKDIR /a
WORKDIR b
WORKDIR c
RUN pwd
```

The output of the final pwd command in this Dockerfile would be /a/b/c.
:::

# Best Practices
<https://docs.docker.com/develop/develop-images/dockerfile_best-practices/>


## Multistage
https://docs.docker.com/build/building/multi-stage/

- With multi-stage builds, you use multiple FROM statements in your Dockerfile. 
- Each FROM instruction can use a different base, and each of them begins a new stage of the build.
- You can selectively copy artifacts from one stage to another, leaving behind everything you don’t want in the final image

### Multistage Examples

:::: {.columns}

::: {.fragment .column width="50%"}
**[Java](https://www.baeldung.com/ops/docker-cache-maven-dependencies)**

```DockerFile
FROM maven:alpine as build
ENV HOME=/usr/app
RUN mkdir -p $HOME
WORKDIR $HOME
ADD . $HOME
RUN mvn package

FROM openjdk:8-jdk-alpine 
COPY --from=build /usr/app/target/single-module-caching-1.0-SNAPSHOT-jar-with-dependencies.jar /app/runner.jar
ENTRYPOINT java -jar /app/runner.jar
```    
::: 

::: {.fragment .column width="50%"}
**[Python](https://medium.com/@ketangangal98/docker-201-multi-stage-builds-for-production-59b1ea98924a)**

```DockerFile
# Build Stage :  [ Step 1 - 3 ]
FROM python:3.10-slim AS build 

# Install Dependencies 
RUN apt-get update && apt-get -y install libpq-dev gcc
COPY ./requirements.txt requirements.txt
RUN pip3 install --no-cache-dir --target=packages -r requirements.txt

# Runtime Stage [ Step 4 - 5 ]
FROM python:3.10-slim AS runtime
# Copying Dependencies from build stage  
COPY --from=build packages /usr/lib/python3.10/site-packages
ENV PYTHONPATH=/usr/lib/python3.10/site-packages

# Security Context 
RUN useradd -m nonroot
USER nonroot

# Env configuration [ Step 6 ]
WORKDIR /app
COPY . .
EXPOSE 8080
ENTRYPOINT ["python","app.py"]
``` 
:::
::::



# To commit or not to commit? {background-color="black" background-image="https://miro.medium.com/v2/resize:fit:1400/format:webp/1*8eIsVx46votEzNqBFvSjpg.jpeg" background-size="100%" background-opacity="0.3"}

<https://medium.com/@lizrice/to-commit-or-not-to-commit-5ab72f9a466e>

:::{.fragment}
**Don’t do it.**

No, really, I know it’s tempting, but if you’re ever going to use this container image again, don’t use commit. 

There are significant downsides to the commit approach:

- You can’t reproduce the image
- You can’t change the base image
:::

:::{.fragment}
Creating a Dockerfile might feel a bit like work, but if you might want this image again, it’s the approach you need to take.
:::